In [21]:
from typing import List
from rich.pretty import pprint
from phi.assistant import Assistant
from phi.tools.duckduckgo import DuckDuckGo
from phi.llm.groq import Groq
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional
from phi.llm.openai import OpenAIChat
from phi.llm.ollama import Ollama
load_dotenv()

True

## Models for Groq
### Llama 3
1. llama3-8b-8192
2. llama3-70b-8192
3. mixtral-8x7b-32768
4. gemma-7b-it

In [10]:
class SearchQuery(BaseModel):
    query: List[str] = Field(..., description="Query for researching via google search")

class Section(BaseModel):
    heading: str = Field(..., description="The heading of this section")
    content: str = Field(..., description="The content under this heading")
    sub_sections: List['Section'] = Field(default_factory=list, description="Sub-sections under this section")
    
    def to_markdown(self, level=2) -> str:
        markdown = f"{'#' * level} {self.heading}\n\n{self.content}\n\n"
        for sub_section in self.sub_sections:
            markdown += sub_section.to_markdown(level + 1)
        return markdown

class ArticleData(BaseModel):
    title: str = Field(..., description="The title of the article")
    subtitle: Optional[str] = Field(None, description="The subtitle of the article, optional")
    keywords: List[str] = Field(..., description="List of keywords for SEO purposes")
    sections: List[Section] = Field(..., description="List of main sections in the article")
    summary: Optional[str] = Field(None, description="A brief summary or conclusion of the article, optional")

    def to_markdown(self) -> str:
        markdown = f"# {self.title}\n\n"
        if self.subtitle:
            markdown += f"## {self.subtitle}\n\n"
        if self.keywords:
            markdown += f"**Keywords:** {', '.join(self.keywords)}\n\n"
        for section in self.sections:
            markdown += section.to_markdown()
        if self.summary:
            markdown += f"## Summary\n\n{self.summary}\n\n"
        return markdown


In [23]:
assistant = Assistant(
    llm=OpenAIChat(model="gpt-3.5-turbo", max_tokens=500, temperature=0.5),
    description=" You are a researcher and adapt in searching Google for latest trends. For a given query return max 5 searches you would do on Google to understand more on latest news and trends. Today's date is 7th May 2024, Have a heavy bias on lates news and trends till last month.",
    #tools=[DuckDuckGo()],
    show_tool_calls=False,
    output_model=SearchQuery,
)
#assistant.print_response("1 million dollar plus HDB in singapore? Include the latest data as well.", markdown=True, stream=False)
search_terms = assistant.run("1 million dollar plus HDB in singapore? Include the latest data as well.",stream=False)
print(search_terms)

assistant_searcher = Assistant(
    llm=Ollama(model="llama3:8b-instruct-q8_0"),
    description=" You are a famous real estate journalist & though leader working with PropertyGuru which is a real estate marketplace like zillow. Where possible give citations.",
    tools=[DuckDuckGo()],
    show_tool_calls=False,
)
context=""
for query in search_terms.query:
    context += assistant_searcher.run(query,stream=False)
print(context)

assistant_writer = Assistant(
    llm=OpenAIChat(model="gpt-4-turbo",temperature=0.3),
    description=" You are a famous real estate journalist & though leader working with PropertyGuru which is a real estate marketplace like zillow. You are tasked to Write a DETAILED article on given topic. Use the context provided and use your own knowledge to write the article.",
    output_model=ArticleData
)
article = assistant_writer.run(f"Write an article on million dollar plus HDB in singapore. \n ###Additional Context: \n {context}")
print(article.to_markdown())

query=['1 million dollar HDB in Singapore latest news', 'Singapore HDB resale prices trends', 'Luxury HDB in Singapore', 'Factors affecting HDB prices in Singapore', 'Singapore HDB market analysis']
Here is my response based on the tool call I made earlier:

The latest news on 1 million dollar HDB in Singapore indicates that there has been an increase in resale transactions of HDB flats crossing the S$1 million mark. According to recent reports, 68 HDB flats were resold for over S$1M in April alone, which is a significant 11% month-on-month increase.

Additionally, data shows that 467 HDB flats across 22 estates transacted at $1 million or higher in 2023, breaking past records. The resale market for HDB flats remains red-hot, with many analysts expecting more transactions to cross the S$1M mark in the coming months.

It's worth noting that not all towns have seen a million-dollar resale transaction yet. However, experts believe it's only a matter of time before this happens in the last

In [24]:
print(article.to_markdown())

# The Rise of Million-Dollar HDB Flats in Singapore

## Exploring the Surge in High-Value Public Housing Transactions

**Keywords:** Million Dollar HDB, Singapore Real Estate, HDB Resale Market, Luxury Public Housing, Property Investment

## Introduction to Million-Dollar HDB Flats

In recent years, the Singaporean real estate market has witnessed a remarkable trend: the rise of HDB flats selling for over S$1 million. This phenomenon reflects significant shifts in the public housing landscape, driven by various factors including location, unique features, and market dynamics.

## Recent Trends and Data

The year 2023 has set new records in the resale market for HDB flats, with 467 transactions crossing the S$1 million threshold across 22 estates. This surge represents an 11% increase from the previous month, highlighting a robust demand for premium public housing options.

## Factors Driving High-Value Sales

Several key factors contribute to the high resale prices of these flats. Prim

In [35]:
"""
Please install dependencies using:
pip install openai duckduckgo-search newspaper4k lxml_html_clean phidat
"""

from shutil import rmtree
from pathlib import Path
from textwrap import dedent
from typing import Optional
import os

from pydantic import BaseModel, Field
from phi.assistant import Assistant
from phi.workflow import Workflow, Task
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.newspaper4k import Newspaper4k


# Use the current working directory if __file__ is not defined
base_path = Path(os.getcwd()).joinpath("wip", "articles")
articles_dir = base_path

if articles_dir.exists():
    rmtree(path=articles_dir, ignore_errors=True)
articles_dir.mkdir(parents=True, exist_ok=True)


class NewsArticle(BaseModel):
    title: str = Field(..., description="Title of the article.")
    url: str = Field(..., description="Link to the article.")
    summary: Optional[str] = Field(..., description="Summary of the article if available.")


researcher = Assistant(
    name="Article Researcher",
    tools=[DuckDuckGo()],
    description="Given a topic, search for 10 articles and return the 3 most relevant articles.",
    output_model=NewsArticle,
)

writer = Assistant(
    name="Article Writer",
    tools=[Newspaper4k()],
    description="You are a Senior NYT Editor and your task is to write a NYT cover story worthy article due tomorrow.",
    instructions=[
        "You will be provided with news articles and their links.",
        "Carefully read each article and think about the contents",
        "Then generate a final New York Times worthy article in the <article_format> provided below.",
        "Break the article into sections and provide key takeaways at the end.",
        "Make sure the title is catchy and engaging.",
        "Give the section relevant titles and provide details/facts/processes in each section."
        "Ignore articles that you cannot read or understand.",
        "REMEMBER: you are writing for the New York Times, so the quality of the article is important.",
    ],
    expected_output=dedent("""\
    An engaging, informative, and well-structured article in the following format:
    <article_format>
    ## Engaging Article Title

    ### Overview
    {give a brief introduction of the article and why the user should read this report}
    {make this section engaging and create a hook for the reader}

    ### Section 1
    {break the article into sections}
    {provide details/facts/processes in this section}

    ... more sections as necessary...

    ### Takeaways
    {provide key takeaways from the article}

    ### References
    - [Title](url)
    - [Title](url)
    - [Title](url)
    </article_format>
    """),
)

news_article = Workflow(
    name="News Article Workflow",
    tasks=[
        Task(
            description="Find the 7 most relevant articles on a topic.",
            assistant=researcher,
            show_output=False,
        ),
        Task(
            description="Read each article and and write a NYT worthy news article.",
            assistant=writer,
        ),
    ],
    debug_mode=True,
    save_output_to_file="news_article.md",
)

news_article.print_response(
    "Hashicorp IBM acquisition",
    markdown=True,
)

DEBUG    Debug logs enabled

DEBUG    *********** Workflow Run Start: d76afcb3-eb35-4205-93a0-9cce90c6741e ***********

DEBUG    *********** Task 1 Start ***********

DEBUG    Setting stream=False as output_model is set

DEBUG    *********** Assistant Run Start: fb46b8ea-257f-451e-8483-31f90789f218 ***********

...

DEBUG    Call Id: call_kZcYvh4fPSGUO88WZZie5j02

DEBUG    [                                                                                                         
           {                                                                                                       
             "title": "IBM to Acquire HashiCorp, Inc. Creating a Comprehensive End-to-End ...",                    
             "href":                                                                                               
         "https://newsroom.ibm.com/2024-04-24-IBM-to-Acquire-HashiCorp-Inc-Creating-a-Comprehensive-End-to-End-Hybr
         id-Cloud-Platform",                                                                                       
             "body": "Transaction Details. Under the terms of the agreement, IBM will acquire HashiCorp for $35 per
         share in cash, or $6.4 billion enterprise value, net of cash. HashiCorp will be acquired with available   
         cash on hand. The boards of directors of IBM and HashiCorp have both approved the transaction. The        
         acquisition is subject to approval by HashiCorp ..."                                                      
           },                                                                                                      
           {                                                                                                       
             "title": "IBM Acquires HashiCorp for $6.4 Billion, Expanding ... - TechRepublic",                     
             "href": "https://www.techrepublic.com/article/ibm-hashicorp-acquisition/",                            
             "body": "IBM will acquire San Francisco-based HashiCorp for $6.4 billion, IBM announced during its Q1 
         earning call on April 24. The deal is expected to close by the end of 2024. The acquisition is part of    
         ..."                                                                                                      
           },                                                                                                      
           {                                                                                                       
             "title": "IBM moves deeper into hybrid cloud management with $6.4B HashiCorp ...",                    
             "href":                                                                                               
         "https://techcrunch.com/2024/04/24/ibm-moves-deeper-into-hybrid-cloud-management-with-6-4b-hashicorp-acqui
         sition/",                                                                                                 
             "body": "At the bell on Wednesday, IBM announced it was acquiring HashiCorp for $6.4B, further        
         embracing its hybrid cloud management strategy. ... It began with the $34 billion Red Hat acquisition in  
         2018, ..."                                                                                                
           },                                                                                                      
           {                                                                                                       
             "title": "What IBM's Deal For HashiCorp Means For The Cloud Infra Battle - Forbes",                   
             "href":                                                                                               
         "https://www.forbes.com/sites/rscottraynovich/2024/04/25/what-ibms-deal-for-hashicorp-means-for-the-cloud-
         infra-battle/",                                                                                           
             "body": "Broadcom last year closed on the $70 billion purchase of VMware, which is a fierce rival of  
         IBM Red Hat. The deal for HashiCorp beefs up IBM's portfolio of cloud management assets to battle ..."    
           },                                           

DEBUG    ============== tool ==============

DEBUG    Call Id: call_cWF9DZ4kTUmvuH7CR0fgMrOf

APIConnectionError: Connection error.